# pymysql

-   파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
-   Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
-   https://github.com/PyMySQL/PyMySQL/
-   https://pymysql.readthedocs.io/en/latest/


# 설치

-   조건
    -   python version 3.6 이상
    -   mysql version 5.6 이상
-   설치
    -   `pip install pymysql`
    -   `conda install -c conda-forge pymysql`


# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host="DBMS 서버 ip",
                                     port="port번호",
                                     user="계정명",
                                     password="비밀번호",
                                     db="연결할데이터베이스이름",
                                     charset='utf8')
    ```
    - port 번호 기본값: 3306
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
3. Cusror를 이용해 SQL문 전송
    ```python
        cursor.execute("sql문")
    ```
4. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다.
    ```python
    cursor.close()
    connection.close()
    ```


# 예제

## 테이블 생성


In [1]:
create_sql = """
create table customer(
 id  int  auto_increment  primary key,
 name  varchar(20) not null,
 email  varchar(50) not null unique, 
 tall   decimal(5,2), 
 birthday  date,
 created_at  datetime  not null
)"""   # sql 문 마지막에 `;` 은 붙이지 않는다.

In [7]:
import pymysql
try:
    conn = None
    # 1. Database와 연결.
    conn = pymysql.connect(
        host="127.0.0.1",    # DBMS 의 ip(host) : str
        port=3306,            # DBMS의 port 번호: int
        user="playdata",     # username: str
        password="1111",   # password: str
        db="mydb"            #  연결할 Database이름: str
    )  # 연결 성공하면 연결된 DB와 관련 작업할 수있는 기능을 
     #   제공하는 Connection객체를 반환
    print(type(conn)) 
    
    # 2. Connection을 사용해서 Cursor 객체 생성
    #    Cursor: sql 처리를 하는 기능을 제공.( sql 전송하고 처리결과를 받을 때 까지를 관리)
    cursor = conn.cursor()
    print(type(cursor))
    
    # 3. SQL 문 전송  +  처리결과 받기.
    result = cursor.execute(create_sql) 
    print("처리결과: ", result)
finally:
    # 4. 연결닫기(끊기)
    if conn:
        cursor.close() # 4-1. cursor 연결 닫기
        conn.close()  # 4-2. connection 연결 닫기

<class 'pymysql.connections.Connection'>
<class 'pymysql.cursors.Cursor'>


OperationalError: (1050, "Table 'customer' already exists")

## DML

### insert


In [11]:
a = "ㅁ니ㅏㅇ러ㅣㅏㅁㄴ어리ㅏㅁㄴ어리ㅏㅇㅁ너리ㅏㅁㄴ어\리ㅏㅇㄴ머ㅣㅏ러ㅏㅣㅇㅁㄴ"
print(a)

ㅁ니ㅏㅇ러ㅣㅏㅁㄴ어리ㅏㅁㄴ어리ㅏㅇㅁ너리ㅏㅁㄴ어       리ㅏㅇㄴ머ㅣㅏ러ㅏㅣㅇㅁㄴ


In [17]:
sql = "insert into customer (name, email, tall, birthday, created_at) \
 values('이순신', 'lee1@naver.com', 185.23, '2000-09-20', now())"

In [18]:
# connection, cursor 생성 -> with 문으로 작성.
##  manual commit 이 default. 
##  DML(insert/update/delete) 처리 후 commit을 실행해야한다.
# try:
with pymysql.connect(host="127.0.0.1", port=3306, 
                             user="playdata", password="1111", db="mydb") as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql) #insert/update/delete의 경우 변경된 행수를 반환.
        print("결과:", result)
        # commit 처리
        conn.commit()

결과: 1


### Parameterized Query

-   Parameterized Query
    -   SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 list나 tuple로 제공한다.
    -   query문을 쉽게 작성할 수 있는 장점이 있다.


In [23]:
name = input("이름:")
email = input("이메일주소:")

이름: 홍길동
이메일주소: aaaaa@aaa.com


In [24]:
import pymysql
insert_sql = "insert into customer (name, email, tall, birthday, created_at) \
values (%s, %s, %s, %s, now())"

with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(insert_sql, [name, email, 175.23, "2010-01-01"])
        conn.commit()
        print("처리행:", result)

처리행: 1


### Parameterized Query를 이용해 여러개 행 insert

#### for문 사용


# 파이썬 - sql 데이터타입
- str - 문자열타입(char, varchar, text,...)
- int - 정수(tiny int , int, ....)
- float - 실수 (decimal, float, double)
- date - date
- time - time
- datetime - datetime, timestamp

In [30]:
from datetime import date, datetime, time
a = date(2000, 10, 20) # 년, 월, 일
a.year, a.month, a.day
b = datetime(2000,  10, 20, 12, 23)
b.year, b.month, b.day, b.hour, b.minute, b.second

datetime.now()  # 실행시점의 일시 -> datetime (mysql: now())
date.today()      # 실행시점의 날짜 -> date  (mysql: curdate())

datetime.date(2024, 3, 22)

In [42]:
from datetime import date, datetime

datas = [
    # ["name1", "abc1@abc.com", 165, date(2000, 1, 12), datetime.now()],
    ["name2", "def1@abc.com", 175, date(1995, 12, 20), datetime.now()],
    ["name3", "ghi1@abc.com", 185, date(1988, 7, 21), datetime.now()],
]
insert_sql = "insert into customer(name, email, tall, birthday, created_at) \
values(%s, %s, %s, %s, %s)"
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        for data in datas:
            r = cursor.execute(insert_sql, data)
            print(r)
        conn.commit()

1
1


#### executemany() 사용

-   insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.


In [43]:
datas = [
    ["이름1", "abc2@a.com", 165, date(2000, 1, 12), datetime.now()],
    ["이름2", "def2@a.com", 175, date(1995, 12, 20), datetime.now()],
    ["이름1", "ghi2@a.com", 185, date(1988, 7, 21), datetime.now()],
]

In [44]:
insert_sql = "insert into customer(name, email, tall, birthday, created_at) \
values(%s, %s, %s, %s, %s)"
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, datas)
        conn.commit()

In [37]:
print(cnt)  # 모든 sql이 적용된 행수들.

3


### update/delete

-   코딩 절차는 insert 와 동일


In [39]:
update_sql = "update customer set tall=%s where id=%s"
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: "))
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [tall, cust_id])
        print("처리행수: ", result)
        conn.commit()

변경할 키:  182
변경할 고객 ID:  11


처리행수:  1


In [40]:
delete_sql = "delete from customer where tall > %s"

tall = float(input("삭제기준 키:"))

with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(delete_sql, [tall])
        print("처리행수: ", result)
        conn.commit()

삭제기준 키: 170


처리행수:  9


## select (DQL - Data Query Language)

-   조회결과 조회
    -   cursor.execute("select문") 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
-   fetch메소드
    -   **fetchall()**
        -   조회한 모든 행을을 반환
    -   **fetchmany(size=개수)**
        -   지정한 size개수 만큼 반환
    -   **fetchone()**
        -   조회결과 중 첫번째 행만 반환
        -   주로 pk 동등 조건으로 조회한 경우 사용


### fetchall()


In [47]:
sql = "select id, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

조회행수: 6


In [49]:
type(resultset), len(resultset)

(tuple, 6)

In [51]:
resultset

((8, 'name1', Decimal('165.00')),
 (15, 'name2', Decimal('175.00')),
 (16, 'name3', Decimal('185.00')),
 (17, '이름1', Decimal('165.00')),
 (18, '이름2', Decimal('175.00')),
 (19, '이름1', Decimal('185.00')))

### fetchone()


### fetchmany()


### cursor 는 iterable 타입

-   for문에 select 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.


In [52]:
#1행의 name
resultset[0][1]

'name1'

### fetchmanay()

In [53]:
sql = "select * from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchmany(3)

조회행수: 6


In [54]:
len(resultset)

3

In [55]:
resultset

((8,
  'name1',
  'abc1@abc.com',
  Decimal('165.00'),
  datetime.date(2000, 1, 12),
  datetime.datetime(2024, 3, 22, 16, 7, 41)),
 (15,
  'name2',
  'def1@abc.com',
  Decimal('175.00'),
  datetime.date(1995, 12, 20),
  datetime.datetime(2024, 3, 22, 16, 30, 6)),
 (16,
  'name3',
  'ghi1@abc.com',
  Decimal('185.00'),
  datetime.date(1988, 7, 21),
  datetime.datetime(2024, 3, 22, 16, 30, 6)))

### fetchone()

In [60]:
sql = "select * from customer where id = %s" 
cust_id = int(input("조회할 ID: "))
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql, [cust_id])
        print("조회행수:", result)
        resultset = cursor.fetchone()

조회할 ID:  8


조회행수: 1


In [63]:
type(resultset), len(resultset)
resultset

(8,
 'name1',
 'abc1@abc.com',
 Decimal('165.00'),
 datetime.date(2000, 1, 12),
 datetime.datetime(2024, 3, 22, 16, 7, 41))

In [64]:
sql = "select * from customer where id = %s" 
cust_id = int(input("조회할 ID: "))
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb'
                             , cursorclass=pymysql.cursors.DictCursor) as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql, [cust_id])
        print("조회행수:", result)
        resultset = cursor.fetchone()

조회할 ID:  8


조회행수: 1


In [65]:
type(resultset)

dict

In [66]:
resultset

{'id': 8,
 'name': 'name1',
 'email': 'abc1@abc.com',
 'tall': Decimal('165.00'),
 'birthday': datetime.date(2000, 1, 12),
 'created_at': datetime.datetime(2024, 3, 22, 16, 7, 41)}

In [67]:
resultset['name']

'name1'

### cursor 는 iterable

In [71]:
sql = "select * from customer"
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb'
                             , cursorclass=pymysql.cursors.DictCursor) as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        for data in cursor:  # 조회결과를 한행씩 제공 (tuple)
            print(data)
            print('----------------------------')

{'id': 8, 'name': 'name1', 'email': 'abc1@abc.com', 'tall': Decimal('165.00'), 'birthday': datetime.date(2000, 1, 12), 'created_at': datetime.datetime(2024, 3, 22, 16, 7, 41)}
----------------------------
{'id': 15, 'name': 'name2', 'email': 'def1@abc.com', 'tall': Decimal('175.00'), 'birthday': datetime.date(1995, 12, 20), 'created_at': datetime.datetime(2024, 3, 22, 16, 30, 6)}
----------------------------
{'id': 16, 'name': 'name3', 'email': 'ghi1@abc.com', 'tall': Decimal('185.00'), 'birthday': datetime.date(1988, 7, 21), 'created_at': datetime.datetime(2024, 3, 22, 16, 30, 6)}
----------------------------
{'id': 17, 'name': '이름1', 'email': 'abc2@a.com', 'tall': Decimal('165.00'), 'birthday': datetime.date(2000, 1, 12), 'created_at': datetime.datetime(2024, 3, 22, 16, 30, 11)}
----------------------------
{'id': 18, 'name': '이름2', 'email': 'def2@a.com', 'tall': Decimal('175.00'), 'birthday': datetime.date(1995, 12, 20), 'created_at': datetime.datetime(2024, 3, 22, 16, 30, 11)}
----

In [28]:
import customer_db as cdb
cdb.select_customer_by_id(16)

(16,
 'name3',
 'ghi1@abc.com',
 Decimal('185.00'),
 datetime.date(1988, 7, 21),
 datetime.datetime(2024, 3, 22, 16, 30, 6))

In [29]:
# manual commit
# sql 처리하는 함수가 commit 하지 않고 호출 하는 쪽에서 commit/rollback 처리하도록 한다.
# connection 생성(DB연결)/commit->sql 처리 함수 호출하는 쪽에서 생성.
# sql 처리 함수는 connection을 파라미터로 받아서 cursor를 생성.
import pymysql
def insert_customer(conn, name, email, tall, birthday):
    sql = "insert into customer (name, email, tall, birthday, created_at) \
             values (%s, %s, %s, %s, now())"
    with conn.cursor() as cursor:
        return  cursor.execute(sql, [name, email, tall, birthday])
    

def update_customer(conn, cust_id, name, email, tall, birthday):
    sql = "update customer set name=%s, email=%s, tall=%s, birthday=%s where id=%s"
    with conn.cursor() as cursor:
        return cursor.execute(sql, [name, email, tall, birthday, cust_id])

def delete_customer_by_id(conn, cust_id):
    sql = "delete from customer where id=%s"
    with conn.cursor() as cursor:
        return cursor.execute(sql, [cust_id])

def select_customers(conn): # 전체조회
    sql = "select * from customer"
    with conn.cursor() as cursor:
        cursor.execute(sql)
        return cursor.fetchall()



In [31]:
# 업무처리 함수(로직)

with pymysql.connect(host="127.0.0.1", port=3306, user="playdata", password="1111", db='mydb') as conn:
    insert_customer(conn, "이름100", "이메일100", 192.2, '1999-10-01')
    insert_customer(conn, "이름101", "이메일101", 191.2, '1999-10-01')
    insert_customer(conn, "이름102", "이메일102", 190.2, '1999-10-01')
    # conn.rollback() # rollback
    conn.commit()

In [26]:
%%writefile customer_db.py
# autocommit 함수
import pymysql
def insert_customer(name, email, tall, birthday):
    sql = "insert into customer (name, email, tall, birthday, created_at) \
             values (%s, %s, %s, %s, now())"
    with pymysql.connect(host="127.0.0.1", port=3306, user="playdata", password="1111", db='mydb') as conn:
        with conn.cursor() as cursor:
            result = cursor.execute(sql, [name, email, tall, birthday])
            conn.commit()
    return result

def update_customer(cust_id, name, email, tall, birthday):
    sql = "update customer set name=%s, email=%s, tall=%s, birthday=%s where id=%s"
    with pymysql.connect(host="127.0.0.1", port=3306, user="playdata", password="1111", db='mydb') as conn:
        with conn.cursor() as cursor:
            result = cursor.execute(sql, [name, email, tall, birthday, cust_id])
            conn.commit()
    return result

def delete_customer_by_id(cust_id):
    sql = "delete from customer where id=%s"
    with pymysql.connect(host="127.0.0.1", port=3306, user="playdata", password="1111", db='mydb') as conn:
        with conn.cursor() as cursor:
            result = cursor.execute(sql, [cust_id])
            conn.commit()
    return result

def delete_customer_by_name(name):
    sql = "delete from customer where name = %s"
    with pymysql.connect(host="127.0.0.1", port=3306, user="playdata", password="1111", db='mydb') as conn:
        with conn.cursor() as cursor:
            result = cursor.execute(sql, [name])
            conn.commit()
    return result

def select_customer_by_id(cust_id):
    sql = "select * from customer where id = %s"  # PK 조건으로 조회: 결과행 - 1행. => fetechone()
    with pymysql.connect(host="127.0.0.1", port=3306, user="playdata", password="1111", db='mydb') as conn:
        with conn.cursor() as cursor:
            cursor.execute(sql, [cust_id])
            return cursor.fetchone()

def select_customer_by_name(name):
    # 조회조건이 unique 하지 않은 컬럼: 결과행: 0 ~ N행. => fetchall()
    sql = "select * from customer where name = %s" 
    with pymysql.connect(host="127.0.0.1", port=3306, user="playdata", password="1111", db='mydb') as conn:
        with conn.cursor() as cursor:
            cursor.execute(sql, [name])
            return cursor.fetchall()

def select_customer_by_tall_range(start_tall, stop_tall):
    sql = "select * from customer where tall between %s and %s"
    with pymysql.connect(host="127.0.0.1", port=3306, user="playdata", password="1111", db='mydb') as conn:
        with conn.cursor() as cursor:
            cursor.execute(sql, [start_tall, stop_tall])
            return cursor.fetchall()

Writing customer_db.py


In [25]:
select_customer_by_tall_range(150, 200)

((8,
  'name1',
  'abc1@abc.com',
  Decimal('165.00'),
  datetime.date(2000, 1, 12),
  datetime.datetime(2024, 3, 22, 16, 7, 41)),
 (15,
  'name2',
  'def1@abc.com',
  Decimal('175.00'),
  datetime.date(1995, 12, 20),
  datetime.datetime(2024, 3, 22, 16, 30, 6)),
 (16,
  'name3',
  'ghi1@abc.com',
  Decimal('185.00'),
  datetime.date(1988, 7, 21),
  datetime.datetime(2024, 3, 22, 16, 30, 6)),
 (18,
  '이름2',
  'def2@a.com',
  Decimal('175.00'),
  datetime.date(1995, 12, 20),
  datetime.datetime(2024, 3, 22, 16, 30, 11)),
 (21,
  '이름2',
  '새이메일',
  Decimal('180.20'),
  datetime.date(2000, 10, 10),
  datetime.datetime(2024, 3, 25, 9, 40, 6)))

In [23]:
select_customer_by_name('이름2')

((18,
  '이름2',
  'def2@a.com',
  Decimal('175.00'),
  datetime.date(1995, 12, 20),
  datetime.datetime(2024, 3, 22, 16, 30, 11)),
 (21,
  '이름2',
  '새이메일',
  Decimal('180.20'),
  datetime.date(2000, 10, 10),
  datetime.datetime(2024, 3, 25, 9, 40, 6)))

In [18]:
select_customer_by_id(16)

(16,
 'name3',
 'ghi1@abc.com',
 Decimal('185.00'),
 datetime.date(1988, 7, 21),
 datetime.datetime(2024, 3, 22, 16, 30, 6))

In [16]:
delete_customer_by_name("이름1")

2

In [14]:
delete_customer_by_id(20)

1

In [12]:
update_customer(20, "바뀐이름2", "바뀐이메일", 200.20, '2010-11-22')

1

# TODO

다음 함수들을 구현하시오.  
각 함수들은 customer 테이블과 관련해서 CRUD를 처리하는 함수들 입니다.

1. name, email, tall, birthday를 매개변수로 받아서 insert하는 함수. (id는 자동증가, created_at은 실행시점의 일시가 insert되도록 한다.)

In [22]:
insert_customer("이름2", "새이메일", 180.2, '2000-10-10')

1

2. id, name, email, tall, birthday를 매개변수로 받아서 id의 member의 나머지 정보를 update하는 함수. (created_at은 update하지 않는다.)

3. id를 매개변수로 받아서 그 member 를 삭제하는 함수.
4. 이름을 매개변수로 받아서 그 이름의 member를 삭제하는 함수.
5. id를 매개변수로 받아서 그 id의 회원 정보를 조회하여 반환하는 함수.
6. name을 매개변수로 받아서 그 이름이 들어간 회원의 정보를 조회하여 반환하는 함수.
7. birthday를 매개변수로 받아서 그 생일의 회원의 정보를 조회하여 반환하는 함수.
8. tall 값을 두개 받아서 그 범위의 tall인 회원들의 정보를 조회하여 반환하는 함수.

-   insert, update, delete 는 적용된 행의 개수를 반환한다.
-   select 처리 함수는 조회결과를 반환한다.

# TODO

-   hr database의 emp 테이블의 데이터를 다루는 함수들을 구현

1. 직원 정보들(emp의 컬럼들)을 매개변수로 받아 insert 하는 함수

2. 수정할 직원 정보들을 매개변수로 받아 받은 emp_id의 직원의 나머지 컬럼값들을 수정하는 함수

3. emp_id를 매개변수로 받아 그 직원을 삭제하는 함수

4. job을 매개변수로 받아 그 업무를 하는 직원들을 삭제하는 함수

5. 전체 직원정보를 모두 조회해서 반환하는 함수

6. emp_id를 매개변수로 받아 그 직원 정보를 조회해서 반환하는 함수

7. dept_name을 매개변수로 받아서 그 부서에 소속된 직원들의 정보를 조회해서 반환하는 함수

8. salary 범위를 매개변수로 받아 그 범위의 salary를 받는 직원들의 정보를 조회해서 반환하는 함수

9. emp_id를 매개변수로 받아서 그 직원의 커미션이 얼마인지를(salary \* comm_pct) 조회해 반환하는 함수

10. dept_name을 매개변수로 받아서 부서별 급여 통계정보(합계, 최대, 최소, 평균, 표준편차)를 반환하는 함수
